In [18]:
%%capture
!pip install transformers
!pip install datasets
!pip install torchvision
!pip install ipynb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import EarlyStoppingCallback
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report 
from transformers import get_scheduler
from transformers import AdamW
import shutil
import os
import torch
from ipynb.fs.full.eval_metrics import *
hello()

Hello World!


In [2]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [3]:
# before starting, empty the Cuda cache to have more memory space
import gc
gc.collect()
torch.cuda.empty_cache()

In [4]:
path = "../full_datasets/control_group_mentions_data_without_retweets.csv"

In [5]:
def predict_tweets(von_n, bis_n):    
    # Splitting up the tweets to avoid crashing the kernel
    tweets = pd.read_csv(path)
    # make sure no "NA" values are in the text column
    tweets = tweets[tweets['text'].notna()]
    print(len(tweets))
     #400k zu 600k
    von = von_n
    bis = bis_n
    tweets = tweets[von:bis]
    len(tweets)

    # Initialize the Tokenizers for each model
    tokenizer = AutoTokenizer.from_pretrained("EIStakovskii/german_toxicity_classifier_plus_v2")
    tokenizer1 = AutoTokenizer.from_pretrained("german-nlp-group/electra-base-german-uncased")
    tokenizer2 = AutoTokenizer.from_pretrained("deepset/gbert-base")
    # ----- 3. Predict -----#
    # Load test data
    X_test = list(tweets["text"])
    X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)
    X_test_tokenized1 = tokenizer1(X_test, padding=True, truncation=True, max_length=512)
    X_test_tokenized2 = tokenizer2(X_test, padding=True, truncation=True, max_length=512)

    # Create torch dataset
    test_dataset = Dataset(X_test_tokenized)
    test_dataset1 = Dataset(X_test_tokenized1)
    test_dataset2 = Dataset(X_test_tokenized2)

    # Load trained models
    model = AutoModelForSequenceClassification.from_pretrained(f"mox/german_toxicity_classifier_offensive_language_politicians", num_labels=2)
    model1 = AutoModelForSequenceClassification.from_pretrained(f"mox/electra_german_uncased_offensive_language_politicians", num_labels=2)
    model2 = AutoModelForSequenceClassification.from_pretrained(f"mox/gbert_base_offensive_language_politicians", num_labels=2)

    # Define test trainer
    test_trainer = Trainer(model)
    test_trainer1 = Trainer(model1)
    test_trainer2 = Trainer(model2)

    # Make prediction
    print("Trainer 1")
    raw_pred, _, _ = test_trainer.predict(test_dataset)
    print("Trainer 2")
    raw_pred1, _1, _1 = test_trainer1.predict(test_dataset1)
    print("Trainer 3")
    raw_pred2, _2, _2 = test_trainer2.predict(test_dataset2)

    # Preprocess raw predictions
    y_pred = np.argmax(raw_pred, axis=1)
    y_pred1 = np.argmax(raw_pred1, axis=1)
    y_pred2 = np.argmax(raw_pred2, axis=1)

    sum_array = np.add(raw_pred, raw_pred1)
    sum_array = np.add(sum_array, raw_pred2)

    multi = np.multiply(sum_array, 0.5)

    y_pred_soft = np.argmax(multi, axis=1)
    y_pred_soft

    preds = y_pred_soft.tolist()
    # Append model predictions to the dataframe
    tweets["model_predictions"] = preds
    tweets.head(2)
    tweets.to_csv(f"control_group_tweets_predicted_{von}_until_{bis}.csv")
    
    del tweets
    del X_test
    del tokenizer
    del tokenizer1
    del tokenizer2
    del X_test_tokenized
    del X_test_tokenized1
    del X_test_tokenized2
    del test_dataset
    del test_dataset1
    del test_dataset2
    del model
    del model1
    del model2
    del test_trainer
    del test_trainer1
    del test_trainer2
    del raw_pred
    del raw_pred1
    del raw_pred2
    del y_pred
    del y_pred1
    del y_pred2
    del sum_array
    del multi
    del y_pred_soft
    del preds

In [6]:
four_to_six = predict_tweets(400000,600000)

/tmp/ipykernel_1174/303617089.py:3: DtypeWarning: Columns (0,1,3,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv(path)


1534833


No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 200000
  Batch size = 8


Trainer 1


***** Running Prediction *****
  Num examples = 200000
  Batch size = 8


Trainer 2


***** Running Prediction *****
  Num examples = 200000
  Batch size = 8


Trainer 3


In [7]:
six_to_eight = predict_tweets(600000,800000)

/tmp/ipykernel_1174/303617089.py:3: DtypeWarning: Columns (0,1,3,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv(path)


1534833


loading file vocab.txt from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/vocab.txt
loading file tokenizer.json from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/tokenizer_config.json
loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--german-nlp-group--electra-b

Trainer 1


***** Running Prediction *****
  Num examples = 200000
  Batch size = 8


Trainer 2


***** Running Prediction *****
  Num examples = 200000
  Batch size = 8


Trainer 3


In [8]:
eight_to_ten = predict_tweets(800000,1000000)

/tmp/ipykernel_1174/303617089.py:3: DtypeWarning: Columns (0,1,3,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv(path)


1534833


loading file vocab.txt from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/vocab.txt
loading file tokenizer.json from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/tokenizer_config.json
loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--german-nlp-group--electra-b

Trainer 1


***** Running Prediction *****
  Num examples = 200000
  Batch size = 8


Trainer 2


***** Running Prediction *****
  Num examples = 200000
  Batch size = 8


Trainer 3


In [ ]:
ten_to_twelve = predict_tweets(1000000,1200000)

/tmp/ipykernel_1174/303617089.py:3: DtypeWarning: Columns (0,1,3,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv(path)


1534833


loading file vocab.txt from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/vocab.txt
loading file tokenizer.json from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/jovyan/.cache/huggingface/hub/models--EIStakovskii--german_toxicity_classifier_plus_v2/snapshots/6c8e91acf96ceeb7f92386057251e3e988d759e6/tokenizer_config.json
loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--german-nlp-group--electra-b

Trainer 1


In [ ]:
twelve_to_fourteen = predict_tweets(1200000,1400000)

/tmp/ipykernel_521/303617089.py:3: DtypeWarning: Columns (0,1,3,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv(path)


1534833


No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 200000
  Batch size = 8


Trainer 1


***** Running Prediction *****
  Num examples = 200000
  Batch size = 8


Trainer 2


***** Running Prediction *****
  Num examples = 200000
  Batch size = 8


Trainer 3


In [ ]:
fourteen_to_end = predict_tweets(1400000,1534833)

In [13]:
batch_1 = pd.read_csv("control_group_tweets_predicted_1000000_until_1200000.csv")
len(batch_1)

/tmp/ipykernel_409/2125364615.py:1: DtypeWarning: Columns (1,2,9,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  batch_1 = pd.read_csv("control_group_tweets_predicted_1000000_until_1200000.csv")


200000

In [14]:
batch_1.model_predictions.value_counts()

0    167995
1     32005
Name: model_predictions, dtype: int64

In [16]:
del batch_1

# Merging Datasets again

In [20]:
import pandas as pd
import glob

files = glob.glob("../Predictions/*.csv")

In [21]:
df = pd.DataFrame()
for f in files:
    csv = pd.read_csv(f)
    df = df.append(csv)

/tmp/ipykernel_974/3216205998.py:3: DtypeWarning: Columns (2,5,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv(f)
/tmp/ipykernel_974/3216205998.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(csv)
/tmp/ipykernel_974/3216205998.py:3: DtypeWarning: Columns (2,5,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv(f)
/tmp/ipykernel_974/3216205998.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(csv)
/tmp/ipykernel_974/3216205998.py:3: DtypeWarning: Columns (2,3,5,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.read_csv(f)
/tmp/ipykernel_974/3216205998.py:4: FutureWarning: The frame.append method is deprecated and will be removed from p

In [22]:
len(df)

2150444

In [24]:
df = df.drop(["Unnamed: 0.2", "Unnamed: 0.1"], axis=1)
df

,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,quote_count,retweets,replies,likes,mentioned,Runs,Partei,model_predictions
0,1,1435940649572966400.0,CHARLYCAPRI4,1,14.0,NaN,NaN,"@SWagenknecht Sie sind gescheitert ,weil Sie d...",2022-03-31 23:51:42+00:00,0,0,0.0,0.0,['SWagenknecht'],Linke_polis_1,Die Linke,0
1,2,528677411.0,jo711,6,308.0,"Ewiger Revolluzer :-) ,\nBiker,Hundeverrückt.N...",jo,@SWagenknecht Mittlerweile langweilen Sie .\nK...,2022-03-31 23:51:27+00:00,0,0,0.0,0.0,['SWagenknecht'],Linke_polis_1,Die Linke,0
2,3,762265140.0,nimmdenbus,1574,118892.0,Take the bus! Prenez le bus! Mein Herz schlägt...,NaN,@MarvinWendland1 @SWagenknecht @BinBerlinerIn ...,2022-03-31 23:50:40+00:00,0,0,0.0,0.0,"['MarvinWendland1', 'SWagenknecht', 'BinBerlin...",Linke_polis_1,Die Linke,0
3,4,94034298.0,derstaatsanwalt,325,5762.0,Ius vigilantibus scriptum est | er/ihr | rt/fo...,Standort ausgeblendet,"@voglerk @NiemaMovassat Mit Landleben hat das,...",2022-03-31 23:45:33+00:00,0,0,0.0,0.0,"['voglerk', 'NiemaMovassat']",Linke_polis_1,Die Linke,0
4,5,3098650023.0,emerald_fm,99,45077.0,HOL' DIR DEINEN OHRWURM!\n+49-30-92094422\n🐒🐋🐝...,"Prusse Hrouge, Berlin, Germany","@DietmarBartsch Alles Gute zum 63sten noch, He...",2022-03-31 23:45:30+00:00,0,0,0.0,1.0,['DietmarBartsch'],Linke_polis_1,Die Linke,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,3601292,1148916458983964673,DagmarFreitag13,1280,4085.0,MdB 1994-2021 • SPD • Transatlantikerin • Atl...,"Iserlohn, Deutschland",@JoSteiniger Eins so schlecht wie das andere. ...,2022-03-14 12:49:31+00:00,0.0,0.0,0.0,0.0,['JoSteiniger'],CDUCSU_polis_3,CSU/CDU,0
199996,3601294,1148916458983964673,DagmarFreitag13,1280,4085.0,MdB 1994-2021 • SPD • Transatlantikerin • Atl...,"Iserlohn, Deutschland",@JoSteiniger Unmittelbar nach der Flutkatastro...,2022-03-14 12:48:54+00:00,0.0,0.0,1.0,1.0,['JoSteiniger'],CDUCSU_polis_3,CSU/CDU,0
199997,3601295,1470993750252007425,BauingM,1302,10748.0,Leben und leben lassen.\nGlaube nur der Statis...,NaN,@jensspahn Die komplette Co2-Steuer gehört abg...,2022-03-14 12:48:45+00:00,0.0,1.0,0.0,8.0,['jensspahn'],CDUCSU_polis_3,CSU/CDU,0
199998,3601296,825765542816575489,SantorinRh,261,11781.0,🥋🦂🎨❤🛁🛁🛁,NaN,@jensspahn Zum Ersten Mal bin ich der gleichen...,2022-03-14 12:48:10+00:00,0.0,0.0,0.0,1.0,['jensspahn'],CDUCSU_polis_3,CSU/CDU,0


In [25]:
df = df.reset_index()
df

,index,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,quote_count,retweets,replies,likes,mentioned,Runs,Partei,model_predictions
0,0,1,1435940649572966400.0,CHARLYCAPRI4,1,14.0,NaN,NaN,"@SWagenknecht Sie sind gescheitert ,weil Sie d...",2022-03-31 23:51:42+00:00,0,0,0.0,0.0,['SWagenknecht'],Linke_polis_1,Die Linke,0
1,1,2,528677411.0,jo711,6,308.0,"Ewiger Revolluzer :-) ,\nBiker,Hundeverrückt.N...",jo,@SWagenknecht Mittlerweile langweilen Sie .\nK...,2022-03-31 23:51:27+00:00,0,0,0.0,0.0,['SWagenknecht'],Linke_polis_1,Die Linke,0
2,2,3,762265140.0,nimmdenbus,1574,118892.0,Take the bus! Prenez le bus! Mein Herz schlägt...,NaN,@MarvinWendland1 @SWagenknecht @BinBerlinerIn ...,2022-03-31 23:50:40+00:00,0,0,0.0,0.0,"['MarvinWendland1', 'SWagenknecht', 'BinBerlin...",Linke_polis_1,Die Linke,0
3,3,4,94034298.0,derstaatsanwalt,325,5762.0,Ius vigilantibus scriptum est | er/ihr | rt/fo...,Standort ausgeblendet,"@voglerk @NiemaMovassat Mit Landleben hat das,...",2022-03-31 23:45:33+00:00,0,0,0.0,0.0,"['voglerk', 'NiemaMovassat']",Linke_polis_1,Die Linke,0
4,4,5,3098650023.0,emerald_fm,99,45077.0,HOL' DIR DEINEN OHRWURM!\n+49-30-92094422\n🐒🐋🐝...,"Prusse Hrouge, Berlin, Germany","@DietmarBartsch Alles Gute zum 63sten noch, He...",2022-03-31 23:45:30+00:00,0,0,0.0,1.0,['DietmarBartsch'],Linke_polis_1,Die Linke,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150439,199995,3601292,1148916458983964673,DagmarFreitag13,1280,4085.0,MdB 1994-2021 • SPD • Transatlantikerin • Atl...,"Iserlohn, Deutschland",@JoSteiniger Eins so schlecht wie das andere. ...,2022-03-14 12:49:31+00:00,0.0,0.0,0.0,0.0,['JoSteiniger'],CDUCSU_polis_3,CSU/CDU,0
2150440,199996,3601294,1148916458983964673,DagmarFreitag13,1280,4085.0,MdB 1994-2021 • SPD • Transatlantikerin • Atl...,"Iserlohn, Deutschland",@JoSteiniger Unmittelbar nach der Flutkatastro...,2022-03-14 12:48:54+00:00,0.0,0.0,1.0,1.0,['JoSteiniger'],CDUCSU_polis_3,CSU/CDU,0
2150441,199997,3601295,1470993750252007425,BauingM,1302,10748.0,Leben und leben lassen.\nGlaube nur der Statis...,NaN,@jensspahn Die komplette Co2-Steuer gehört abg...,2022-03-14 12:48:45+00:00,0.0,1.0,0.0,8.0,['jensspahn'],CDUCSU_polis_3,CSU/CDU,0
2150442,199998,3601296,825765542816575489,SantorinRh,261,11781.0,🥋🦂🎨❤🛁🛁🛁,NaN,@jensspahn Zum Ersten Mal bin ich der gleichen...,2022-03-14 12:48:10+00:00,0.0,0.0,0.0,1.0,['jensspahn'],CDUCSU_polis_3,CSU/CDU,0


In [26]:
df.to_csv("../full_datasets/mentions_predicted.csv")

In [27]:
df.model_predictions.value_counts()

0    1750146
1     400298
Name: model_predictions, dtype: int64